## Lab. 12

### Solve the following problem using Genetic Algorithms:


Problem: Weighted N-Queen Problem


You are given an N×N chessboard, and each cell of the board has an associated weight. Your task is to find a valid placement of N queens such that the total weight of the queens is maximized, and no two queens threaten each other.





In the traditional N-Queen Problem, the goal is to place N queens on an N×N chessboard in such a way that no two queens threaten each other. In this variation, we introduce weights to the queens and aim to find a placement that maximizes the total weight of the queens while satisfying the constraint of non-threatening positions.


Constraints:

1. There should be exactly one queen in each row and each column.
2. No two queens should be placed in the same diagonal, i.e., they should not threaten each other.
3. The placement should maximize the total weight of the queens.


Representation:

Use a permutation-based representation. Each permutation represents the column position of the queen for each row. 

For example, if N=4, a valid permutation [2, 4, 1, 3] indicates that the queen in the first row is placed in column 2, the queen in the second row is placed in column 4, and so on.


Genetic Algorithm Steps:

1. *Initialization*: Generate an initial population of permutations randomly.

2. *Fitness Evaluation*: Evaluate the fitness of each permutation by calculating the total weight of the queens while considering the non-threatening positions.

3. *Selection*: Select a subset of permutations from the population based on their fitness, using selection techniques like tournament selection or roulette wheel selection.

4. *Crossover*: Perform crossover (recombination) on the selected permutations to create new offspring permutations.

5. *Mutation*: Introduce random changes (mutations) in the offspring permutations to maintain diversity in the population.

6. *Fitness Evaluation for the new individuals*: Evaluate the fitness of the new population.

7. *Form the new population*: Select the surviving individuals based on scores, with chances direct proportional with their performance.

8. Repeat steps 3-7 for a certain number of generations or until a termination condition is met (e.g., a maximum number of iterations or a satisfactory solution is found).


9. *Termination*: Return the best-performing individual (permutation) found as the solution to the problem.

Note: The fitness function used in this problem should calculate the total weight of the queens based on the positions specified by the permutation. Additionally, the fitness function should penalize solutions that violate the non-threatening constraint by assigning a lower fitness score to such permutations.

In [72]:
import random
import time

In [73]:
# Define the parameters
N = 8  # Size of the chessboard
population_size = 200  # Number of individuals in the population
mutation_rate = 0.1  # Probability of mutation

# Define the chessboard weights (randomly generated for this example)
weights = [[random.randint(1, 10) for _ in range(N)] for _ in range(N)]


def calculate_fitness(permutation):
    total_weight = 0
    for row, col in enumerate(permutation):
        total_weight += weights[row][col]
        for i in range(row + 1, N):
            if abs(row - i) == abs(col - permutation[i]):
                total_weight -= 10
    return total_weight


def generate_initial_population():
    population = []
    for _ in range(population_size):
        permutation = random.sample(range(N), N)
        population.append(permutation)
    return population

In [74]:
def selection(population):
    # Tournament selection
    selected = []
    for _ in range(population_size):
        tournament = random.sample(population, 2)
        fitness_values = [calculate_fitness(permutation) for permutation in tournament]
        selected.append(tournament[fitness_values.index(max(fitness_values))])
    return selected

In [101]:
def selection1(population):
    # Roulette wheel selection
    fitness_values = [calculate_fitness(permutation) for permutation in population]

    # Shift fitness values to be non-negative
    min_fitness = min(fitness_values)
    shifted_fitness = [fitness - min_fitness for fitness in fitness_values]

    total_fitness = sum(shifted_fitness)

    # Calculate selection probabilities
    selection_probabilities = [fitness / total_fitness for fitness in shifted_fitness]

    # Perform roulette wheel selection
    selected = []
    for _ in range(population_size):
        spin = random.random()
        cumulative_probability = 0
        for i, probability in enumerate(selection_probabilities):
            cumulative_probability += probability
            if spin <= cumulative_probability:
                selected.append(population[i])
                break

    return selected

In [75]:
import random

def crossover1(parent1, parent2):
    # Position-based crossover
    N = len(parent1)
    child = [-1] * N

    # Create a list of positions to copy from parent1
    positions = random.sample(range(N), N//2)

    # Copy elements from parent1 to child at the selected positions
    for pos in positions:
        child[pos] = parent1[pos]

    # Fill the remaining positions with elements from parent2
    parent2_index = 0
    for i in range(N):
        if child[i] == -1:
            while parent2[parent2_index] in child:
                parent2_index = (parent2_index + 1) % N
            child[i] = parent2[parent2_index]
            parent2_index = (parent2_index + 1) % N

    return child


In [76]:
def crossover(parent1, parent2):
    # Order crossover
    N = len(parent1)
    child = [-1] * N

    # Select a random substring from parent1
    start_pos = random.randint(0, N - 1)
    end_pos = random.randint(start_pos, N - 1)
    for i in range(start_pos, end_pos + 1):
        child[i] = parent1[i]

    # Fill the remaining positions with elements from parent2
    parent2_index = 0
    for i in range(N):
        if child[i] == -1:
            while parent2[parent2_index] in child:
                parent2_index = (parent2_index + 1) % N
            child[i] = parent2[parent2_index]
            parent2_index = (parent2_index + 1) % N

    return child

In [77]:
def crossover2(parent1, parent2):
    # Heuristic crossover
    N = len(parent1)
    child = [-1] * N

    # Copy elements from parent1 that are not present in parent2
    for i in range(N):
        if parent1[i] not in parent2:
            child[i] = parent1[i]

    # Fill the remaining positions with elements from parent2
    for i in range(N):
        if child[i] == -1:
            child[i] = parent2[i]

    return child

In [78]:
def mutation(individual):
    # Swap mutation
    if random.random() < mutation_rate:
        pos1, pos2 = random.sample(range(N), 2)
        individual[pos1], individual[pos2] = individual[pos2], individual[pos1]

    # Fix duplicate values
    duplicates = set([x for x in individual if individual.count(x) > 1])
    while duplicates:
        for i in range(N):
            if individual[i] in duplicates:
                available_values = set(range(N)).difference(individual)
                if available_values:
                    individual[i] = random.choice(list(available_values))
                else:
                    individual[i] = random.randint(0, N-1)
        duplicates = set([x for x in individual if individual.count(x) > 1])

    return individual

In [79]:
def mutation1(individual):
    # Swap mutation
    N = len(individual)
    mutation_rate = calculate_mutation_rate()

    # Perform mutation on each gene
    for i in range(N):
        if random.random() < mutation_rate:
            individual[i] = generate_random_gene(N, individual)

    return individual


def generate_random_gene(N, individual):
    available_values = set(range(N)).difference(individual)
    if available_values:
        return random.choice(list(available_values))
    else:
        return random.randint(0, N-1)


def calculate_mutation_rate():
    # You can define your own rules for calculating the mutation rate here
    return 0.01  # Default mutation rate

In [80]:
def genetic_algorithm():
    # Generate the initial population
    population = generate_initial_population()

    # Evolution loop
    for _ in range(100):
        # Select individuals for reproduction
        selected_population = selection(population)

        # Create new offspring population through crossover and mutation
        offspring_population = []
        while len(offspring_population) < population_size:
            parent1, parent2 = random.sample(selected_population, 2)
            child = crossover(parent1, parent2)
            child = mutation(child)
            offspring_population.append(child)

        # Calculate fitness for the new population
        fitness_values = [calculate_fitness(permutation) for permutation in offspring_population]

        # Form the new population based on fitness
        population = [permutation for _, permutation in sorted(zip(fitness_values, offspring_population), reverse=True)]

    # Return the best-performing individual (permutation)
    return population[0]

In [88]:
def genetic_algorithm1():
    # Generate the initial population
    population = generate_initial_population()

    # Evolution loop
    for _ in range(100):
        # Select individuals for reproduction
        selected_population = selection(population)

        # Create new offspring population through crossover and mutation
        offspring_population = []
        while len(offspring_population) < population_size:
            parent1, parent2 = random.sample(selected_population, 2)
            child = crossover2(parent1, parent2)
            child = mutation1(child)
            offspring_population.append(child)

        # Calculate fitness for the new population
        fitness_values = [calculate_fitness(permutation) for permutation in offspring_population]

        # Form the new population based on fitness
        population = [permutation for _, permutation in sorted(zip(fitness_values, offspring_population), reverse=True)]

    # Return the best-performing individual (permutation)
    return population[0]

In [82]:
def genetic_algorithm2():
    # Generate the initial population
    population = generate_initial_population()

    # Evolution loop
    for _ in range(100):
        # Select individuals for reproduction
        selected_population = selection(population)

        # Create new offspring population through crossover and mutation
        offspring_population = []
        while len(offspring_population) < population_size:
            parent1, parent2 = random.sample(selected_population, 2)
            child = crossover1(parent1, parent2)
            child = mutation(child)
            offspring_population.append(child)

        # Calculate fitness for the new population
        fitness_values = [calculate_fitness(permutation) for permutation in offspring_population]

        # Form the new population based on fitness
        population = [permutation for _, permutation in sorted(zip(fitness_values, offspring_population), reverse=True)]

    # Return the best-performing individual (permutation)
    return population[0]

In [97]:
def genetic_algorithm3():
    # Generate the initial population
    population = generate_initial_population()

    # Evolution loop
    for _ in range(100):
        # Select individuals for reproduction
        selected_population = selection1(population)

        # Create new offspring population through crossover and mutation
        offspring_population = []
        while len(offspring_population) < population_size:
            parent1, parent2 = random.sample(selected_population, 2)
            child = crossover2(parent1, parent2)
            child = mutation1(child)
            offspring_population.append(child)

        # Calculate fitness for the new population
        fitness_values = [calculate_fitness(permutation) for permutation in offspring_population]

        # Form the new population based on fitness
        population = [permutation for _, permutation in sorted(zip(fitness_values, offspring_population), reverse=True)]

    # Return the best-performing individual (permutation)
    return population[0]

In [107]:
# Function to print the chessboard with queens' positions
def print_chessboard(positions):
    for row in range(N):
        line = ""
        for col in range(N):
            if positions[row] == col:
                line += "Q "
            else:
                line += "- "
        print(line)
    print()

    
start_time = time.time()
# Run the genetic algorithm and print the solution
solution = genetic_algorithm()
end_time = time.time()  # Stop the timer
execution_time = end_time - start_time

print("Tournament Selection + Order crossover + Swap mutation\n")
print("Execution time:", execution_time, "seconds")
print("Best solution:")
print_chessboard(solution)
print("Fitness:", calculate_fitness(solution))

Tournament Selection + Order crossover + Swap mutation

Execution time: 0.533653736114502 seconds
Best solution:
- - - - - Q - - 
Q - - - - - - - 
- - - - Q - - - 
- Q - - - - - - 
- - - - - - - Q 
- - Q - - - - - 
- - - - - - Q - 
- - - Q - - - - 

Fitness: 51


In [108]:
# Function to print the chessboard with queens' positions
def print_chessboard(positions):
    for row in range(N):
        line = ""
        for col in range(N):
            if positions[row] == col:
                line += "Q "
            else:
                line += "- "
        print(line)
    print()

    
start_time = time.time()
# Run the genetic algorithm and print the solution
solution = genetic_algorithm1()
end_time = time.time()  # Stop the timer
execution_time = end_time - start_time

print("Tournament selection + Heuristic crossover + Swap mutation\n")
print("Execution time:", execution_time, "seconds")
print("Best solution:")
print_chessboard(solution)
print("Fitness:", calculate_fitness(solution))

Tournament selection + Heuristic crossover + Swap mutation

Execution time: 0.4058375358581543 seconds
Best solution:
- - - - - Q - - 
- - - - - Q - - 
Q - - - - - - - 
- - - - Q - - - 
- - - - - - - Q 
- - Q - - - - - 
- - - - - - Q - 
- Q - - - - - - 

Fitness: 45


In [109]:
# Function to print the chessboard with queens' positions
def print_chessboard(positions):
    for row in range(N):
        line = ""
        for col in range(N):
            if positions[row] == col:
                line += "Q "
            else:
                line += "- "
        print(line)
    print()

    
start_time = time.time()
# Run the genetic algorithm and print the solution
solution = genetic_algorithm2()
end_time = time.time()  # Stop the timer
execution_time = end_time - start_time

print("Tournament selection + Position-based crossover + Adaptive mutation\n")
print("Execution time:", execution_time, "seconds")
print("Best solution:")
print_chessboard(solution)
print("Fitness:", calculate_fitness(solution))

Tournament selection + Position-based crossover + Adaptive mutation

Execution time: 0.5631518363952637 seconds
Best solution:
- Q - - - - - - 
- - - - Q - - - 
Q - - - - - - - 
- - - - - Q - - 
- - - - - - - Q 
- - Q - - - - - 
- - - - - - Q - 
- - - Q - - - - 

Fitness: 53


In [110]:
# Function to print the chessboard with queens' positions
def print_chessboard(positions):
    for row in range(N):
        line = ""
        for col in range(N):
            if positions[row] == col:
                line += "Q "
            else:
                line += "- "
        print(line)
    print()

    
start_time = time.time()
# Run the genetic algorithm and print the solution
solution = genetic_algorithm3()
end_time = time.time()  # Stop the timer
execution_time = end_time - start_time

print("Roulette selection + Heuristic crossover + Swap mutation\n")
print("Execution time:", execution_time, "seconds")
print("Best solution:")
print_chessboard(solution)
print("Fitness:", calculate_fitness(solution))

Roulette selection + Heuristic crossover + Swap mutation

Execution time: 0.43787264823913574 seconds
Best solution:
- - - - - Q - - 
Q - - - - - - - 
- - - - - - Q - 
- - - - Q - - - 
- - Q - - - - - 
- - - - - - - Q 
- - - Q - - - - 
Q - - - - - - - 

Fitness: 45
